In [1]:
%reset -f
%matplotlib inline

import time

import cv2
import numpy as np
import matplotlib.pyplot as plt


EXIT_KEY = 27
N_POINTS = 100

In [3]:
# Capture webcam
cap = cv2.VideoCapture(0)

# Set video buffer size to 1 to prevent buffer filling up
cap.set(cv2.CAP_PROP_BUFFERSIZE, 1)

while True:
    # Get current frame
    ret, frame = cap.read()

    # Get list of points matching certain criteria
    #
    # goodFeaturesToTrack() uses 1 channel images to choose points,
    # so we convert frame to grayscale
    params = {
        'maxCorners': N_POINTS,
        'qualityLevel': 0.3,
        'minDistance': 7,
        'blockSize': 7
    }
    
    pts = cv2.goodFeaturesToTrack(
        cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY),
        mask = None,
        **params
    )

    # Generate N_POINTS random colors => (R, G, B) x N_POINTS
    color = np.random.randint(0, 255, (N_POINTS, 3))

    # Insert a circle in each point
    for i, pt in enumerate(pts):
            a, b = pt.ravel()
            frame = cv2.circle(frame, (a, b), 5, color[i].tolist(), -1)
        
    # Mirror image because OpenCV is flipped by default
    flipped = cv2.flip(frame, 1)
    
    # Show webcam
    cv2.imshow("Video", flipped)

    # Close window if EXIT_KEY was pressed
    key = cv2.waitKey(30)
    
    if key == EXIT_KEY:
        break

cap.release()
cv2.destroyAllWindows()